<h3>Semantic Search</h3>

In [4]:
#import libraries
import weaviate
import cohere

In [6]:
#Api cohere key. obtain from cohere.com
cohere_api_key = "Keeb1E63nVYR0dPVIXb4xJWJ4SJau6TAo28rGppJ"

In [7]:
#create a cohere client object.
co = cohere.Client(cohere_api_key)

In [9]:
#create a vector database object
#connect to the weaviate demo database containing 10M wikipedia vectors

auth_config = weaviate.auth.AuthApiKey(api_key="76320a90-53d8-42bc-b41d-678647c6672e")
client = weaviate.Client(
    url = "https://cohere-demo.weaviate.network/",
    auth_client_secret=auth_config,
additional_headers={
    "X-Cohere-Api-Key": cohere_api_key,
    }

)

C:\Users\Thilini\anaconda3\Lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [14]:
#Keyword Function
def keyword_search(query, results_lang='en', num_results=3):
    properties = ["text", "title", "url"]

    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }

    responce = (
    client.query.get("Articles", properties)
    .with_bm25(
    query=query
    )
    .with_where(where_filter)
    .with_limit(num_results)
    .do()
    )
    result = responce['data']['Get']['Articles']
    return result

In [15]:
#Example usage
query = "What is the highest mountain peak in the world?"
top_documents = keyword_search(query)

In [16]:
top_documents

[{'text': "The Polish scientist and explorer Count Paul Edmund Strzelecki conducted surveying work in the Australian Alps in 1839 and, led by his two Aboriginal guides Charlie Tarra and Jackie, became the first European to ascend Australia's highest peak, which he named Mount Kosciuszko in honour of the Polish patriot Tadeusz Kościuszko.European explorers penetrated deeper into the interior in the 1840s in a quest to discover new lands for agriculture or answer scientific enquiries. The German scientist Ludwig Leichhardt led three expeditions in northern Australia in this decade, sometimes with the help of Aboriginal guides, identifying the grazing potential of the region and making important discoveries in the fields of botany and geology. He and his party disappeared in 1848 while attempting to cross the continent from east to west. Edmund Kennedy led an expedition into what is now far-western Queensland in 1847 before being speared by Aborigines in the Cape York Peninsula in 1848.",

In [17]:
# Example usage 2 
query1 = "What is the capital of United State of America?"
top_document2 = keyword_search(query1)

In [18]:
top_document2

[{'text': "North America outside the zone of Spanish settlement was a contested area in the 17th century. Spain had founded small settlements in Florida and Georgia but nowhere near the size of those in New Spain or the Caribbean islands. France, The Netherlands, and Great Britain held several colonies in North America and the West Indies from the 17th century, 100 years after the Spanish and Portuguese established permanent colonies. The British colonies in North America were founded between 1607 (Virginia) and 1733 (Georgia). The Dutch explored the east coast of North America and began founding settlements in what they called New Netherland (now New York State.). France colonized what is now Eastern Canada, founding Quebec City in 1608. France's loss in the Seven Years' War resulted in the transfer of New France to Great Britain. The Thirteen Colonies, in lower British North America, rebelled against British rule in 1775, largely due to the taxation that Great Britain was imposing on

<h3>Search using Dence Retrival and Vector Database</h3>

In [19]:
# Dense retrival function
def dense_retrieval(query, results_lang='en', num_results=3):

    nearText = {"concepts": [query]}
    properties = ["text", "title", "url", "_additional {distance}"]

    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }

    responce = (
    client.query
    .get("Articles", properties)
    .with_near_text(nearText)
    .with_where(where_filter)
    .with_limit(num_results)
    .do()
    )
    result = responce['data']['Get']['Articles']
    return result

In [21]:
top_documents = dense_retrieval(query)

In [23]:
top_documents

[{'_additional': {'distance': -150.58865},
  'text': "Heights of mountains are typically measured above sea level. Using this metric, Mount Everest is the highest mountain on Earth, at . There are at least 100 mountains with heights of over above sea level, all of which are located in central and southern Asia. The highest mountains above sea level are generally not the highest above the surrounding terrain. There is no precise definition of surrounding base, but Denali, Mount Kilimanjaro and Nanga Parbat are possible candidates for the tallest mountain on land by this measure. The bases of mountain islands are below sea level, and given this consideration Mauna Kea ( above sea level) is the world's tallest mountain and volcano, rising about from the Pacific Ocean floor.",
  'title': 'Mountain',
  'url': 'https://en.wikipedia.org/wiki?curid=37754'},
 {'_additional': {'distance': -150.0647},
  'text': 'Until 1852, Kangchenjunga was assumed to be the highest mountain in the world, but ca

In [24]:
top_documents2 = dense_retrieval(query1)

In [25]:
top_documents2

[{'_additional': {'distance': -148.66464},
  'text': "In 1785, the assembly of the Congress of the Confederation made New York City the national capital shortly after the war. New York was the last capital of the U.S. under the Articles of Confederation and the first capital under the Constitution of the United States. New York City as the U.S. capital hosted several events of national scope in 1789—the first President of the United States, George Washington, was inaugurated; the first United States Congress and the Supreme Court of the United States each assembled for the first time; and the United States Bill of Rights was drafted, all at Federal Hall on Wall Street. In 1790, New York surpassed Philadelphia as the nation's largest city. At the end of that year, pursuant to the Residence Act, the national capital was moved to Philadelphia.",
  'title': 'New York City',
  'url': 'https://en.wikipedia.org/wiki?curid=645042'},
 {'_additional': {'distance': -147.85652},
  'text': "New Yor

<h3>Search using Dense retrieval, Rerank and Vector Database</h3>

In [29]:
def rerank_responses(query, responses, num_responses=2):
    reranked_responses = co.rerank(
        model = 'rerank-english-v2.0',
        query = query,
        documents = responses,
        top_n = num_responses,
    )
    return reranked_responses

In [33]:
texts = [result.get('text') for result in top_documents]
reranked_text = rerank_responses(query, texts)

In [34]:
for i, result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{result}")

i:0
('id', '7c9853f0-5a13-49ae-ad20-119edc1357c3')
i:1
('results', [RerankResponseResultsItem(document=None, index=1, relevance_score=0.92892635), RerankResponseResultsItem(document=None, index=2, relevance_score=0.876423)])
i:2
('meta', ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None))


In [35]:
texts = [result.get('text') for result in top_document2]
reranked_text1 = rerank_responses(query, texts)

In [36]:
for i, result in enumerate(reranked_text1):
    print(f"i:{i}")
    print(f"{result}")

i:0
('id', '9deb345e-5dd9-4669-912b-b2f8734529a1')
i:1
('results', [RerankResponseResultsItem(document=None, index=1, relevance_score=0.23881637), RerankResponseResultsItem(document=None, index=2, relevance_score=0.12506539)])
i:2
('meta', ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None))
